In [0]:
#The only issue was the return filtered_key_val
# one way to fix is to simply remove this line
# second method is to encapsulate the second for loop in a function definition as you can see below 
from collections import defaultdict

text_file = sc.textFile("/FileStore/tables/Assignment_1/Question_1.txt")
counts = text_file.flatMap(lambda line: line.split(" ")).map(lambda word:(word, 1)).collect()
key_val = defaultdict(int)
for item in counts:
  key = item[0]
  val = item[1]
  key_val[key] += int(val)

## make it into a function so that way the return code works
def test (key_val):
  filtered_key_val = dict()
  for k, v in key_val.items():
    if v >= 100:
      filtered_key_val[k] = v
  return filtered_key_val

filtered_key_val=test(key_val)



##### 1. How can we preprocess the textual contents in the files? 

*First step is to load the csv using spark.read.csv then create an RDD. *
*Second, use the higher order function map and use lambda to select the Body column -> column 8. and Title column*
*Third, filter out None or null rows, and only take the rows that has < p > tag using beautiful soup *
*Lastly, strip out all the puncioations and noise and stop words from each line then split by space*

##### 2. What are the most frequent keywords in the textual contents of each programming language? 

*After cleaning the data from noise we will have a list of words, now simply convert it to key,val pairs using the map function.*
*Then, use reduceByKey() function to calculate the frequency of words*

##### 3. What percentage of questions in each programming language has accepted answers?
*Create another RDD and use AcceptedAnswerId column inorder to find the percentage. Simply count the rows that are digits and divide it by the total count of the RDD*

##### 4. What types of questions are asked for each programming languages?
*Take title column which describes the question asked. First, split title by ?, then filter to check if theres a why,what, or how in the title, else the question is considered other*

In [0]:
##Loading java.csv
java_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Java.csv",inferSchema=True,header=True,multiLine=True,escape='"')

java_RDD_title=java_df.select('Title').rdd.flatMap(list)
def counter (lis):
  dic = {'how':0, 'what':0, 'why':0,'other':0}
  for row in lis:
    row.split()
    if ('how'in row):
       dic['how']+=1
    elif ('what'in row):
       dic['what']+=1
    elif ('why'in row):
       dic['why']+=1
    else:
      dic['other']+=1
  return dic
  
java_title=java_df.select('Title').rdd.flatMap(list).collect()
dic=counter(java_title)
print(dic)

{'how': 1960, 'what': 267, 'why': 258, 'other': 47515}

In [0]:
dic=counter(java_RDD_title)
print(dic)


{'how': 1960, 'what': 267, 'why': 258, 'other': 47515}

In [0]:
from bs4 import BeautifulSoup
import re
stop_words = sc.textFile("/FileStore/tables/Assignment_1/stopwords.txt").collect()


In [0]:

##Loading java.csv
java_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Java.csv",inferSchema=True,header=True,multiLine=True,escape='"')
java_RDD=java_df.select('Body').rdd.flatMap(list)
java_RDD_title=java_df.select('Title').rdd.flatMap(list)
## preprocess the textual content 
##Body 
java_RDD_body=java_RDD.map(lambda x: str(BeautifulSoup(x).find('p')))
java_RDD_body=java_RDD_body.map(lambda x: re.sub('<p>','',x))
java_RDD_body=java_RDD_body.map(lambda x: re.sub('</p>','',x))
java_filtered=java_RDD_body.map(lambda x: re.sub(r'[^\w]', ' ', x))
java_words=java_filtered.flatMap(lambda line: line.split()).filter(lambda x: not x.isdigit())
java_words=java_words.filter(lambda x: x.lower() not in stop_words )
##title
java_filtered_title=java_RDD_title.map(lambda x: re.sub(r'[^\w]', ' ', x))
java_words_title=java_filtered_title.flatMap(lambda line: line.split()).filter(lambda x: not x.isdigit())
java_words_title=java_words_title.filter(lambda x: x.lower() not in stop_words )
## Creating key,val pair for each word then counting freq 
#Body
java_pairs=java_words.map(lambda x: (x,1))
java_freq=java_pairs.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()
#Title
java_pairs_title=java_words_title.map(lambda x: (x,1))
java_freq_title=java_pairs_title.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()

##calculating % of answered questions
java_accedpted_df=java_df.select('AcceptedAnswerId')
java_total_count=java_accedpted_df.count()
java_accedpted_numeric_df=java_accedpted_df.filter(java_accedpted_df.AcceptedAnswerId >0)
java_count_of_numeric=java_accedpted_numeric_df.count()

## Findindin types of question
java_RDD_title=java_df.select('Title','Id').rdd
java_RDD_title=java_RDD_title.map(lambda x: (x[1],x[0]))
java_RDD_title=java_RDD_title.map(lambda x:(x[0],x[1].split('?')))
java_RDD_title=java_RDD_title.map(lambda x:(x[0],list(filter(None, x[1]))))
java_RDD_title=java_RDD_title.map(lambda x:(x[0], re.sub(r'[^\w]', ' ', x[1][0])))

java_what=java_RDD_title.map(lambda x:(x[0],x[1].split())).filter(lambda x : ('what' in x[1] or 'What' in x[1] or 'whats' in x[1] or 'Whats' in x[1])) 
java_what=java_what.filter(lambda x : ('how' not in [1] and 'How' not in [1]) or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0]) ) 
java_what=java_what.filter(lambda x : ('why' not in [1] and 'Why' not in [1])or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0])).map(lambda x: (x[0]," ".join(x[1])))

java_why=java_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('why' in x[1] or 'Why'in x[1])) 
java_why=java_why.filter(lambda x : ('how' not in x[1] and 'How' not in x[1])or ('why' in x[1][0] or 'Why'in x[1][0])) 
java_why=java_why.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('why' in x[1][0] or 'Why'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

java_how=java_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('how' in x[1] or 'How'in x[1])) 
java_how=java_how.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('how' in x[1][0] or 'How'in x[1][0])) 
java_how=java_how.filter(lambda x : ('why' not in x[1] and 'Why' not in x[1])or ('how' in x[1][0] or 'How'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

java_combined=java_what.union(java_why)
java_combined=java_how.union(java_combined)
java_other=java_RDD_title.subtractByKey(java_combined)

java_what=java_what.map(lambda x: x[1]).count()
java_why=java_why.map(lambda x: x[1]).count()
java_how=java_how.map(lambda x: x[1]).count()
java_other=java_other.map(lambda x: x[1]).count()


In [0]:
##Loading py.csv
py_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Python.csv",inferSchema=True,header=True,multiLine=True,escape='"')
py_RDD=py_df.select('Body').rdd.flatMap(list)
py_RDD_title=py_df.select('Title').rdd.flatMap(list)
## preprocess the textual content 
##Body 
py_RDD_body=py_RDD.map(lambda x: str(BeautifulSoup(x).find('p')))
py_RDD_body=py_RDD_body.map(lambda x: re.sub('<p>','',x))
py_RDD_body=py_RDD_body.map(lambda x: re.sub('</p>','',x))
py_filtered=py_RDD_body.map(lambda x: re.sub(r'[^\w]', ' ', x))
py_words=py_filtered.flatMap(lambda line: line.split()).filter(lambda x: not x.isdigit())
py_words=py_words.filter(lambda x: x.lower() not in stop_words )
##title
py_filtered_title=py_RDD_title.map(lambda x: re.sub(r'[^\w]', ' ', x))
py_words_title=py_filtered_title.flatMap(lambda line: line.split()).filter(lambda x: len(x)>2).filter(lambda x: not x.isdigit())
py_words_title=py_words_title.filter(lambda x: x.lower() not in stop_words )
## Creating key,val pair for each word then counting freq 
#Body
py_pairs=py_words.map(lambda x: (x,1))
py_freq=py_pairs.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()
#Title
py_pairs_title=py_words_title.map(lambda x: (x,1))
py_freq_title=py_pairs_title.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()

##calculating % of answered questions
py_accedpted_df=py_df.select('AcceptedAnswerId')
py_total_count=py_accedpted_df.count()
py_accedpted_numeric_df=py_accedpted_df.filter(py_accedpted_df.AcceptedAnswerId >0)
py_count_of_numeric=py_accedpted_numeric_df.count()

## Findindin types of question
py_RDD_title=py_df.select('Title','Id').rdd
py_RDD_title=py_RDD_title.map(lambda x: (x[1],x[0]))
py_RDD_title=py_RDD_title.map(lambda x:(x[0],x[1].split('?')))
py_RDD_title=py_RDD_title.map(lambda x:(x[0],list(filter(None, x[1]))))
py_RDD_title=py_RDD_title.map(lambda x:(x[0], re.sub(r'[^\w]', ' ', x[1][0])))

py_what=py_RDD_title.map(lambda x:(x[0],x[1].split())).filter(lambda x : ('what' in x[1] or 'What' in x[1] or 'whats' in x[1] or 'Whats' in x[1])) 
py_what=py_what.filter(lambda x : ('how' not in [1] and 'How' not in [1]) or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0]) ) 
py_what=py_what.filter(lambda x : ('why' not in [1] and 'Why' not in [1])or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0])).map(lambda x: (x[0]," ".join(x[1])))

py_why=py_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('why' in x[1] or 'Why'in x[1])) 
py_why=py_why.filter(lambda x : ('how' not in x[1] and 'How' not in x[1])or ('why' in x[1][0] or 'Why'in x[1][0])) 
py_why=py_why.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('why' in x[1][0] or 'Why'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

py_how=py_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('how' in x[1] or 'How'in x[1])) 
py_how=py_how.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('how' in x[1][0] or 'How'in x[1][0])) 
py_how=py_how.filter(lambda x : ('why' not in x[1] and 'Why' not in x[1])or ('how' in x[1][0] or 'How'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

py_combined=py_what.union(py_why)
py_combined=py_how.union(py_combined)
py_other=py_RDD_title.subtractByKey(py_combined)

py_what=py_what.map(lambda x: x[1]).count()
py_why=py_why.map(lambda x: x[1]).count()
py_how=py_how.map(lambda x: x[1]).count()
py_other=py_other.map(lambda x: x[1]).count()


In [0]:
##Loading js.csv
js_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Javascript.csv",inferSchema=True,header=True,multiLine=True,escape='"')
js_RDD=js_df.select('Body').rdd.flatMap(list)
js_RDD_title=js_df.select('Title').rdd.flatMap(list)
## preprocess the textual content 
##Body 
js_RDD_body=js_RDD.map(lambda x: str(BeautifulSoup(x).find('p')))
js_RDD_body=js_RDD_body.map(lambda x: re.sub('<p>','',x))
js_RDD_body=js_RDD_body.map(lambda x: re.sub('</p>','',x))
js_filtered=js_RDD_body.map(lambda x: re.sub(r'[^\w]', ' ', x))
js_words=js_filtered.flatMap(lambda line: line.split()).filter(lambda x: not x.isdigit())
js_words=js_words.filter(lambda x: x.lower() not in stop_words )
##title
js_filtered_title=js_RDD_title.map(lambda x: re.sub(r'[^\w]', ' ', x))
js_words_title=js_filtered_title.flatMap(lambda line: line.split()).filter(lambda x: not x.isdigit())
js_words_title=js_words_title.filter(lambda x: x.lower() not in stop_words )
## Creating key,val pair for each word then counting freq 
#Body
js_pairs=js_words.map(lambda x: (x,1))
js_freq=js_pairs.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()
#Title
js_pairs_title=js_words_title.map(lambda x: (x,1))
js_freq_title=js_pairs_title.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()

##calculating % of answered questions
js_accedpted_df=js_df.select('AcceptedAnswerId')
js_total_count=js_accedpted_df.count()
js_accedpted_numeric_df=js_accedpted_df.filter(js_accedpted_df.AcceptedAnswerId >0)
js_count_of_numeric=js_accedpted_numeric_df.count()

## Findindin types of question
js_RDD_title=js_df.select('Title','Id').rdd
js_RDD_title=js_RDD_title.map(lambda x: (x[1],x[0]))
js_RDD_title=js_RDD_title.map(lambda x:(x[0],x[1].split('?')))
js_RDD_title=js_RDD_title.map(lambda x:(x[0],list(filter(None, x[1]))))
js_RDD_title=js_RDD_title.map(lambda x:(x[0], re.sub(r'[^\w]', ' ', x[1][0])))

js_what=js_RDD_title.map(lambda x:(x[0],x[1].split())).filter(lambda x : ('what' in x[1] or 'What' in x[1] or 'whats' in x[1] or 'Whats' in x[1])) 
js_what=js_what.filter(lambda x : ('how' not in [1] and 'How' not in [1]) or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0]) ) 
js_what=js_what.filter(lambda x : ('why' not in [1] and 'Why' not in [1])or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0])).map(lambda x: (x[0]," ".join(x[1])))

js_why=js_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('why' in x[1] or 'Why'in x[1])) 
js_why=js_why.filter(lambda x : ('how' not in x[1] and 'How' not in x[1])or ('why' in x[1][0] or 'Why'in x[1][0])) 
js_why=js_why.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('why' in x[1][0] or 'Why'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

js_how=js_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('how' in x[1] or 'How'in x[1])) 
js_how=js_how.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('how' in x[1][0] or 'How'in x[1][0])) 
js_how=js_how.filter(lambda x : ('why' not in x[1] and 'Why' not in x[1])or ('how' in x[1][0] or 'How'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

js_combined=js_what.union(js_why)
js_combined=js_how.union(js_combined)
js_other=js_RDD_title.subtractByKey(js_combined)

js_what=js_what.map(lambda x: x[1]).count()
js_why=js_why.map(lambda x: x[1]).count()
js_how=js_how.map(lambda x: x[1]).count()
js_other=js_other.map(lambda x: x[1]).count()


In [0]:
print("----Java csv file analysis-----")
print("Most frequent words in Java Body column are")
print(java_freq[:10])
print("Most frequent words in Java Title column are")
print(java_freq_title[:10])
print("Percentage of answered questions in Java is: {:.2f}%".format(java_count_of_numeric/java_total_count*100))
print("Types of question in Java What={}, Why={}, How={}, and Other={}".format(java_what,java_why,java_how,java_other))
print("\n----Python csv file analysis-----")
print("Most frequent words Body column in Python are")
print(py_freq[:10])
print("Most frequent words in Title column in Python are")
print(py_freq_title[:10])
print("Percentage of answered questions in Python is: {:.2f}%".format(py_count_of_numeric/py_total_count*100))
print("Types of question in Python What={}, Why={}, How={}, and Other={}".format(py_what,py_why,py_how,py_other))
print("\n----JavaScript csv file analysis-----")
print("Most frequent words in Body column in JavaScript are")
print(js_freq[:10])
print("Most frequent words in Title column JavaScript are")
print(js_freq_title[:10])
print("Percentage of answered questions in JavaScript is: {:.2f}%".format(js_count_of_numeric/js_total_count*100))
print("Types of question in JavaScript What={}, Why={}, How={}, and Other={}".format(js_what,js_why,js_how,js_other))


----Java csv file analysis-----
Most frequent words in Java Body column are
[(39903, 'code'), (15891, 'am'), (11198, 'using'), (8053, 'trying'), (7422, 'java'), (7225, 'Java'), (6999, 'file'), (6115, 'class'), (5617, 'strong'), (5460, 'application')]
Most frequent words in Java Title column are
[(8516, 'Java'), (4986, 'java'), (3677, 'using'), (2536, 'file'), (2184, 'class'), (2041, 'Android'), (1912, 'Spring'), (1875, 'method'), (1576, 'error'), (1293, 'data')]
Percentage of answered questions in Java is: 50.09%
Types of question in Java What=1190, Why=1489, How=10247, and Other=37098

----Python csv file analysis-----
Most frequent words Body column in Python are
[(39931, 'code'), (15430, 'am'), (10668, 'using'), (10049, 'trying'), (9742, 'python'), (7167, 'file'), (6782, 'Python'), (6749, 'data'), (4912, 'list'), (4900, 'following')]
Most frequent words in Title column in Python are
[(10224, 'Python'), (7510, 'python'), (4068, 'using'), (3125, 'list'), (2902, 'file'), (2086, 'data'), (2068, 'Django'), (2004, 'function'), (1840, 'error'), (1667, 'string')]
Percentage of answered questions in Python is: 51.10%
Types of question in Python What=860, Why=1251, How=10997, and Other=36921

----JavaScript csv file analysis-----
Most frequent words in Body column in JavaScript are
[(35221, 'code'), (14172, 'am'), (10704, 'using'), (8509, 'trying'), (7277, 'page'), (5982, 'function'), (5612, 'data'), (5577, 'href'), (5101, 'com'), (4758, 'js')]
Most frequent words in Title column JavaScript are
[(3944, 'javascript'), (3608, 'using'), (3419, 'Javascript'), (3141, 'function'), (2797, 'js'), (2449, 'JavaScript'), (2175, 'jQuery'), (2133, 'array'), (2063, 'value'), (2043, 'data')]
Percentage of answered questions in JavaScript is: 55.30%
Types of question in JavaScript What=814, Why=1187, How=10435, and Other=37593